In [2]:
! pip install datasets transformers accelerate
! pip install evaluate
import evaluate
import transformers

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
SUPERGLUE_TASKS = ["boolq","cb","copa","multirc","record","rte","wic","wsc.fixed" ]
task = "boolq"
model_checkpoint = "bert-base-uncased"
batch_size = 16
import datasets
actual_task = "cb" if task == "copa" else task
dataset = datasets.load_dataset("super_glue", actual_task, trust_remote_code=True)

In [4]:
metric = evaluate.load('super_glue', actual_task)

In [6]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])

,question,passage,idx,label
0,are airsoft guns legal in new york state,"Legal issues in airsoft -- During 1987 in New York City, more than 1400 toy imitation weapons involved in criminal acts were seized by New York City police; approximately 80 percent higher from the previous four years. On the basis of legislative intent dealing with the increasing volume of criminal acts in correlation with toy imitation weapons, New York City introduced new guidelines regulating the manufacture, importation, distribution, and sale of such imitation weapons. New York City requires that all realistic toy or imitation firearms be made of clear or brightly colored plastics. Furthermore, New York City makes possession of any pistol or rifle or similar instrument in which the propelling force is a spring or air unlawful without a license. See New York City Administrative Code § 10-131(b) and New York City Administrative Code § 10-131(g)(1)(a).",2014,False
1,do all business have to offer health insurance,"Health insurance in the United States -- However, in a 2007 analysis, the Employee Benefit Research Institute concluded that the availability of employment-based health benefits for active workers in the US is stable. The ``take-up rate,'' or percentage of eligible workers participating in employer-sponsored plans, has fallen somewhat, but not sharply. EBRI interviewed employers for the study, and found that others might follow if a major employer discontinued health benefits. Effective by January 1, 2014, the Patient Protection and Affordable Care Act will impose a $2000 per employee tax penalty on employers with over 50 employees who do not offer health insurance to their full-time workers. (In 2008, over 95% of employers with at least 50 employees offered health insurance.) On the other hand, public policy changes could also result in a reduction in employer support for employment-based health benefits.",5929,False
2,is the united states located in the northern hemisphere,"North America -- The term Northern America refers to the northern-most countries and territories of North America: the United States, Bermuda, St. Pierre and Miquelon, Canada and Greenland. Although the term does not refer to a unified region, Middle America--not to be confused with the Midwestern United States--groups the regions of Mexico, Central America, and the Caribbean.",5621,True
3,is a person's education level included in phi,"Protected health information -- Protected health information (PHI) under the US law is any information about health status, provision of health care, or payment for health care that is created or collected by a Covered Entity (or a Business Associate of a Covered Entity), and can be linked to a specific individual. This is interpreted rather broadly and includes any part of a patient's medical record or payment history.",4658,False
4,are there such things as four leaf clovers,"Four-leaf clover -- The four-leaf clover is a rare variation of the common three-leaf clover. According to traditional superstition, such clovers bring good luck, though it is not clear when or how that superstition got started. The earliest mention of ``Fower-leafed or purple grasse'' is from 1640 and simply says that it was kept in gardens because it was ``good for the purples in children or others''. A description from 1869 says that four-leaf clovers were ``gathered at night-time during the full moon by sorceresses, who mixed it with vervain and other ingredients, while young girls in search of a token of perfect happiness made quest of the plant by day''. The first reference to luck might be from an 11-year-old girl, who wrote in an 1877 letter to St. Nicholas Magazine, ``Did the fairies ever whisper in your ear, that a four-leaf clover brought good luck to the finder?''",8863,True
5,is calgary the largest city in north america,"Calgary -- The city had a population of 1,239,220 in 2016, making it Alberta's largest city and Canada's third-largest municipality. Also

In [6]:
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'accuracy': 0.5625}

In [7]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [8]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [9]:
task_to_keys = {
    "boolq": ("question", "passage"),
    "cb": ("premise", "hypothesis"),
    "copa": ("premise", "choice1"), 
    "multirc": ("question", "paragraph"),
    "record": ("passage", "query"),
    "rte": ("sentence1", "sentence2"),
    "wic": ("sentence1", "sentence2"),  
    "wsc.fixed": ("text", None)
}
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
preprocess_function(dataset['train'][:5])

Sentence 1: do iran and afghanistan speak the same language
Sentence 2: Persian language -- Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.


{'input_ids': [[101, 2079, 4238, 1998, 7041, 3713, 1996, 2168, 2653, 102, 4723, 2653, 1011, 1011, 4723, 1006, 1013, 100, 1010, 1011, 1130, 29681, 2078, 1013, 1007, 1010, 2036, 2124, 2011, 2049, 2203, 16585, 2213, 2521, 5332, 1006, 1291, 25573, 17149, 29824, 24830, 2521, 5332, 1006, 1042, 29678, 23432, 29692, 29715, 5332, 23432, 1007, 1006, 4952, 1007, 1007, 1010, 2003, 2028, 1997, 1996, 2530, 7726, 4155, 2306, 1996, 11424, 1011, 7726, 3589, 1997, 1996, 11424, 1011, 2647, 2653, 2155, 1012, 2009, 2003, 3952, 5287, 1999, 4238, 1010, 7041, 1006, 3985, 2124, 2004, 18243, 2072, 2144, 3845, 1007, 1010, 1998, 23538, 1006, 3985, 2124, 2004, 11937, 4478, 3211, 2144, 1996, 3354, 3690, 1007, 1010, 1998, 2070, 2060, 4655, 2029, 7145, 2020, 4723, 3686, 8384, 1998, 2641, 2112, 1997, 3618, 4238, 1012, 2009, 2003, 2517, 1999, 1996, 4723, 12440, 1010, 1037, 6310, 8349, 1997, 1996, 5640, 5896, 1010, 2029, 2993, 7964, 2013, 1996, 28934, 12440, 1012, 102], [101, 2079, 2204, 3520, 8486, 5794, 4277, 4047, 22

In [10]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
metric_name = "f1" if task == "multirc" else "f1" if task == "record" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task in ["cb", "rte", "boolq", "wic", "wsc.fixed"]:
        predictions = np.argmax(predictions, axis=1)
    elif task == "stsb":
        predictions = predictions[:, 0] 
    elif task in ["multirc", "record"]:
        predictions = np.argmax(predictions, axis=-1) 
    elif task == "copa":
        predictions = np.argmax(predictions, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
validation_key = {
    "boolq": "validation",
    "cb": "validation",
    "copa": "validation",
    "multirc": "validation",
    "record": "validation",
    "rte": "validation",
    "wic": "validation",
    "wsc.fixed": "validation",
}.get(task, "validation")
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

D:\anaconda3\lib\site-packages\transformers\training_args.py:1570: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\DELL\AppData\Local\Temp\ipykernel_20144\2915367881.py:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.644500,0.588926,0.697248
2,0.513900,0.568860,0.718349
3,0.369500,0.704560,0.715902
4,0.263200,0.955945,0.726300
5,0.183700,1.065963,0.724771


TrainOutput(global_step=2950, training_loss=0.3567445451122219, metrics={'train_runtime': 47638.6766, 'train_samples_per_second': 0.989, 'train_steps_per_second': 0.062, 'total_flos': 7023126275745060.0, 'train_loss': 0.3567445451122219, 'epoch': 5.0})

In [14]:
trainer.evaluate()

{'eval_loss': 0.9559453725814819,
 'eval_accuracy': 0.7262996941896025,
 'eval_runtime': 918.5024,
 'eval_samples_per_second': 3.56,
 'eval_steps_per_second': 0.223,
 'epoch': 5.0}

In [15]:
trainer.push_to_hub()

events.out.tfevents.1732287648.DESKTOP-VEQ2CRL:   0%|          | 0.00/363 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/crzhao/bert-base-uncased-finetuned-boolq/commit/4f190cf4d0085e0a8fa33a790d4f7c39be4708fd', commit_message='End of training', commit_description='', oid='4f190cf4d0085e0a8fa33a790d4f7c39be4708fd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/crzhao/bert-base-uncased-finetuned-boolq', endpoint='https://huggingface.co', repo_type='model', repo_id='crzhao/bert-base-uncased-finetuned-boolq'), pr_revision=None, pr_num=None)

In [ ]:
! pip install datasets transformers accelerate
! pip install evaluate
import evaluate
import transformers
from huggingface_hub import notebook_login
notebook_login()
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16
import datasets
actual_task = "mnli" if task == "mnli-mm" else task
dataset = datasets.load_dataset("glue", actual_task)
metric = evaluate.load('glue', actual_task)
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(dataset["train"])
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
preprocess_function(dataset['train'][:5])
encoded_dataset = dataset.map(preprocess_function, batched=True)
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()
trainer.evaluate()